In [7]:
from model import get_global_model
import tensorflow as tf
import numpy as np
import cv2
import pandas as pd
from data import build_dataset, build_dataset_fusion, build_dataset_attention
from matplotlib import pyplot as plt
from datetime import datetime

In [8]:
TRAIN_CSV = "data/data/processed_attention_train.csv"
VALID_CSV = "data/data/processed_attention_valid.csv"
global_model = get_global_model()
local_model = get_global_model()
global_model.load_weights("model/checkpoint.ckpt")
local_model.load_weights("model/AG-CNN-FINAL/checkpoint.ckpt")

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
resnet50 (Model)             (None, 7, 7, 2048)        23587712  
_________________________________________________________________
global_average_pooling2d_4 ( (None, 2048)              0         
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 2049      
Total params: 23,589,761
Trainable params: 23,536,641
Non-trainable params: 53,120
_________________________________________________________________
None
Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
resnet50 (Model)             (None, 7, 7, 2048)        23587712  
_________________________________________________________________
global_average_pooling2d_5 ( (None, 2048)              0         
__________

In [9]:
IMG_SHAPE = [224, 224, 3]
res_net_global = tf.keras.applications.ResNet50(weights='imagenet', include_top=False, input_shape=IMG_SHAPE)
res_net_global.set_weights(global_model.layers[0].get_weights())
res_net_global.trainable = False
for layer in res_net_global.layers:
    layer._name = layer._name + "_global"

In [10]:
res_net_local = tf.keras.applications.ResNet50(weights='imagenet', include_top=False, input_shape=IMG_SHAPE)
res_net_local.set_weights(local_model.layers[0].get_weights())
res_net_local.trainable = False
for layer in res_net_global.layers:
    layer._name = layer._name + "_local"

In [11]:
global_average_layer_global = tf.keras.layers.GlobalAveragePooling2D()
global_average_layer_local = tf.keras.layers.GlobalAveragePooling2D()

In [14]:
model = tf.keras.Sequential([res_net_global, global_average_layer_global])
train_dataset = build_dataset(TRAIN_CSV).prefetch(64).batch(32)
pooling_1 = model.predict(train_dataset)

In [15]:
model = tf.keras.Sequential([res_net_local, global_average_layer_local])
train_dataset = build_dataset_attention(TRAIN_CSV).prefetch(64).batch(32)
pooling_2 = model.predict(train_dataset)

In [16]:
np.savetxt('global_pooling2_final.txt', pooling_1)
np.savetxt('local_pooling2_final.txt', pooling_2)

In [17]:
model = tf.keras.Sequential([res_net_global, global_average_layer_global])
valid_dataset = build_dataset(VALID_CSV).prefetch(64).batch(32)
pooling_1 = model.predict(valid_dataset)
np.savetxt('global_pooling_valid2_final.txt', pooling_1)

In [18]:
model = tf.keras.Sequential([res_net_local, global_average_layer_local])
valid_dataset = build_dataset_attention(VALID_CSV).prefetch(64).batch(32)
pooling_2 = model.predict(valid_dataset)
np.savetxt('local_pooling_valid2_final.txt', pooling_2)